In my Capstone project I analyse InstaCart Online Grocery Baskets. The main question that this project aims to address is how to optimize the product offerings of retailers by identifying customer behavior. By predicting if a customer has a healthy product basket based on the product names and clustering customers based on their product baskets, this project aims to provide insights into how retail should approach next best offer. The goal is to increase profits for retailers by better understanding and targeting the needs and preferences of their customers.


__Please note: this is notebook 5 of 5.__


In this notebook, I performed a market basket analysis by aisles to provide retailes understanding which aisles it is better to place together. To achieve this, I utilized the Apriori algorithm and association rules. 

In [19]:
import pandas as pd

In [29]:
import session_info
session_info.show()

In [20]:
df_1 = pd.read_csv("/Users/evgenijkucukov/Desktop/Brainstation/Capstone/order_prodact_name_final")

In [21]:
df_drop = df_1.loc[(df['aisle'] == 'fresh vegetables') | (df['aisle'] == 'fresh fruits') | (df['aisle'] =='packaged vegetables fruits')]

In [22]:
df_1 = df_1.drop(df_drop.index)

In [23]:
# create dataframe
basket_df_1 = df_1.loc[:,['order_id','aisle']]

In [24]:
# create series
basket_series_1 = basket_df_1.groupby('order_id')['aisle'].apply(list)

In [25]:
from mlxtend.preprocessing import TransactionEncoder

# Transform our basket series into a transaction matrix
te = TransactionEncoder()
transaction_matrix_1 = te.fit_transform(basket_series_1, sparse=True)

# Convert to dataframe
transaction_df_1 = pd.DataFrame.sparse.from_spmatrix(transaction_matrix_1, 
                                                  columns = te.columns_)
transaction_df_1.head()

,air fresheners candles,asian foods,baby accessories,baby bath body care,baby food formula,bakery desserts,baking ingredients,baking supplies decor,beauty,beers coolers,...,spreads,tea,tofu meat alternatives,tortillas flat bread,trail mix snack mix,trash bags liners,vitamins supplements,water seltzer sparkling water,white wines,yogurt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,1


Let's create a dataframe that contains frequently aisles. To achieve this, I will only consider aisles that are present in more than 5% of orders. 

In [26]:
freq_itemsets_1 = apriori(transaction_df_1, min_support=0.05, use_colnames=True, max_len = 3)
freq_itemsets_1.sort_values('support', ascending=False)

,support,itemsets
46,0.257758,(yogurt)
33,0.241866,(packaged cheese)
28,0.227594,(milk)
45,0.208872,(water seltzer sparkling water)
8,0.179715,(chips pretzels)
...,...,...
53,0.054329,"(yogurt, eggs)"
57,0.053832,"(water seltzer sparkling water, packaged cheese)"
29,0.053747,(missing)
52,0.052166,"(packaged cheese, eggs)"


Lets identify aisles which aisles should be located near each other in the stores and website.

In [28]:
rules_1 = association_rules(freq_itemsets_1, metric="lift", min_threshold=1 )
rules_1.sort_values('lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
14,(packaged cheese),(lunch meat),0.241866,0.110456,0.050491,0.208757,1.889960,0.023776,1.124236
15,(lunch meat),(packaged cheese),0.110456,0.241866,0.050491,0.457117,1.889960,0.023776,1.396496
2,(packaged cheese),(bread),0.241866,0.166583,0.062476,0.258310,1.550639,0.022186,1.123673
3,(bread),(packaged cheese),0.166583,0.241866,0.062476,0.375047,1.550639,0.022186,1.213105
0,(milk),(bread),0.227594,0.166583,0.057995,0.254820,1.529690,0.020082,1.118410
1,(bread),(milk),0.166583,0.227594,0.057995,0.348148,1.529690,0.020082,1.184941
18,(milk),(yogurt),0.227594,0.257758,0.088485,0.388787,1.508339,0.029821,1.214374
19,(yogurt),(milk),0.257758,0.227594,0.088485,0.343289,1.508339,0.029821,1.176173
10,(packaged cheese),(eggs),0.241866,0.149993,0.052166,0.215680,1.437937,0.015888,1.083751
11,(eggs),(packaged cheese),0.149993,0.241866,0.052166,0.347788,1.437937,0.015888,1.162404


The analysis revealed that customers tend to purchase packaged cheese together with lunch meat, and packaged cheese together with bread. Other common associations include milk and bread, as well as yogurt and milk. These insights can be used to optimize product placement and promotions in-store and on the website, potentially increasing sales and improving the overall customer experience.